In [36]:
import logging
from flask import Flask
from flask_ask import Ask, statement, question, context

import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

In [37]:
###
# framework set up
###

# add debug logs
# log = logging.getLogger()
# log.addHandler(logging.StreamHandler())
# log.setLevel(logging.DEBUG)
logging.getLogger("flask_ask").setLevel(logging.DEBUG)

# initialise flask-ask
app = Flask(__name__)
ask = Ask(app, "/")

###
# database set up
###

dynamodb = boto3.resource("dynamodb", region_name="eu-west-2", endpoint_url="https://dynamodb.eu-west-2.amazonaws.com")
table = dynamodb.Table("Conversation") # the key is device_id

Loading variable profile from defaults.
Loading variable profile from defaults.
Loading variable profile from defaults.
Loading variable profile from defaults.
Loading variable ca_bundle from defaults.
Loading variable ca_bundle from defaults.
Loading variable ca_bundle from defaults.
Loading variable ca_bundle from defaults.
Loading variable profile from defaults.
Loading variable profile from defaults.
Loading variable profile from defaults.
Loading variable profile from defaults.
Event choose-service-name: calling handler <function handle_service_name_alias at 0x08DEBAE0>
Event choose-service-name: calling handler <function handle_service_name_alias at 0x08DEBAE0>
Event choose-service-name: calling handler <function handle_service_name_alias at 0x08DEBAE0>
Event choose-service-name: calling handler <function handle_service_name_alias at 0x08DEBAE0>
Event creating-client-class.dynamodb: calling handler <function add_generate_presigned_url at 0x08DCEA50>
Event creating-client-class.dy

In [38]:
###
# database utils
###

# get data from dynomodb
def get_item(key): # key - device id
    key_dict = {"device_id": key}
    try:
        response = table.get_item(Key=key_dict)
    except ClientError as e:
        print(e.response["Error"]["Message"])
    else:
        item = response["Item"]
        return item

# put/update data to dynamodb
def update_item(key, intent_id, intent_name, corpus_name):
    key_dict = {"device_id": key}
    expression ="set intent_id = :a, intent_name = :b, corpus_name = :c"
    expression_values = {":a": intent_id, ":b": intent_name, ":c": corpus_name,}
    try:
        response = table.update_item(Key=key_dict, UpdateExpression=expression, 
                                     ExpressionAttributeValues=expression_values,
                                     ReturnValues="UPDATED_NEW")
    except ClientError as e:
        print(e.response["Error"]["Message"])

# delete data from dynamodb
def delete_item(key):
    key_dict = {"device_id": key}
    try:
        response = table.delete_item(Key=key_dict)
    except ClientError as e:
        print(e.response["Error"]["Message"])

In [2]:
###
# functional utils - for keywords function
###

def extract_keywords(raw_sentence): # always returns a list
    import re
    pattern = re.compile(r"\(([a-zA-Z]+)\)")
    if pattern.search(raw_sentence):
        return pattern.findall(raw_sentence)
    else:
        return []
    
def ignore_keywords(raw_sentence):
    import re
    pattern = re.compile(r"\(([a-zA-Z]+)\)")
    if pattern.search(raw_sentence):
        raw_sentence = raw_sentence.replace("(", "")
        raw_sentence = raw_sentence.replace(")", "")
    return raw_sentence

In [7]:
# s = "Hi, how are you, I am fine, (hello) think you! and you? (world)!"
# s1 = "Hi, how are you?"
# ignore_keywords(s1)

'Hi, how are you?'

In [43]:
# update_item("asasa", "0", "restaurant_start_intent", "restaurant_corpus")
# get_item("asasa")
# delete_item("asasa")

Calling dynamodb:delete_item with {'TableName': 'Conversation', 'Key': {'device_id': 'asasa'}}
Calling dynamodb:delete_item with {'TableName': 'Conversation', 'Key': {'device_id': 'asasa'}}
Calling dynamodb:delete_item with {'TableName': 'Conversation', 'Key': {'device_id': 'asasa'}}
Calling dynamodb:delete_item with {'TableName': 'Conversation', 'Key': {'device_id': 'asasa'}}
Event provide-client-params.dynamodb.DeleteItem: calling handler <function copy_dynamodb_params at 0x0904B108>
Event provide-client-params.dynamodb.DeleteItem: calling handler <function copy_dynamodb_params at 0x0904B108>
Event provide-client-params.dynamodb.DeleteItem: calling handler <function copy_dynamodb_params at 0x0904B108>
Event provide-client-params.dynamodb.DeleteItem: calling handler <function copy_dynamodb_params at 0x0904B108>
Event before-parameter-build.dynamodb.DeleteItem: calling handler <bound method TransformationInjector.inject_condition_expressions of <boto3.dynamodb.transform.TransformationI

Signature:
496903c8b7da9ae5ce35079955bf23ecaaae1f99c1e6021c0a71b41d577a12e9
Signature:
496903c8b7da9ae5ce35079955bf23ecaaae1f99c1e6021c0a71b41d577a12e9
Signature:
496903c8b7da9ae5ce35079955bf23ecaaae1f99c1e6021c0a71b41d577a12e9
Signature:
496903c8b7da9ae5ce35079955bf23ecaaae1f99c1e6021c0a71b41d577a12e9
Sending http request: <PreparedRequest [POST]>
Sending http request: <PreparedRequest [POST]>
Sending http request: <PreparedRequest [POST]>
Sending http request: <PreparedRequest [POST]>
"POST / HTTP/1.1" 200 2
"POST / HTTP/1.1" 200 2
"POST / HTTP/1.1" 200 2
"POST / HTTP/1.1" 200 2
Response headers: {'server': 'Server', 'date': 'Mon, 09 Apr 2018 15:54:40 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '38UGU0C3EROEPSDADMNH4HMDF7VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}
Response headers: {'server': 'Server', 'date': 'Mon, 09 Apr 2018 15:54:40 GMT', 'content-type': 'application/x-amz-json-1.0', 'con

In [53]:
###
# model of corpus
###

class Corpus:
    def __init__(self, corpus_filename):
        self.meta_data = {}
        self.data = {}
        
        self.load_corpus(corpus_filename)
        
    def load_corpus(self, corpus_filename):
        with open(corpus_filename) as f:
            raw_data = []
            for line in f:
                raw_data.append(line)
        raw_data = [line.split("::")[-1].strip().replace("\\n", "\n") for line in raw_data]
        
        for i in range(0, len(raw_data), 7): # each conversational must have 7 lines
            self.meta_data[str(int(i/7))] = {}
            self.meta_data[str(int(i/7))]["intent_id"] =  str(int(i/7))
            self.meta_data[str(int(i/7))]["intent_name"] =  raw_data[i].split(",")[1].strip()
            self.meta_data[str(int(i/7))]["corpus_name"] =  raw_data[i].split(",")[-1].strip()
            
            self.data[str(int(i/7))] = {}
            self.data[str(int(i/7))]["user_response"] =  raw_data[i+1]
            self.data[str(int(i/7))]["alexa_response"] =  raw_data[i+2]
            self.data[str(int(i/7))]["card_text"] = raw_data[i+3]
            self.data[str(int(i/7))]["context"] = raw_data[i+4]
            self.data[str(int(i/7))]["img_url"] = raw_data[i+5]
            i += 7
        
    def get_meta_data(self, intent_id): # return dict
            return self.meta_data[intent_id]
            
    def get_data(self, intent_id):
            return self.data[intent_id] # return dict
        
    def get_end_id(self):
        return len(corpus.meta_data)

In [55]:
# corpus = Corpus("restaurant_corpus")
# corpus.get_data("1")["alexa_response"].format("fish and chips")
# print(corpus.get_data("2")["card_text"])
# len(corpus.meta_data)

'Okay, here you is your food.'

In [ ]:
###
# main application
###

@ask.launch
def welcome_response():
    card_title = "Welcome!"
    
    card_content = "Welcome to teach me skill. Which scenario do you want to learn?\n"
    card_content += "\n"
    card_content += "You could say:\n"
    card_content += "1. Order food\n"
    card_content += "\n"
    card_content += "----------\n"
    card_content += "To end the skill: 'Alexa, exit/stop'\n"
    card_content += "To resume the conversation: 'Alexa, ask teachme to continue'\n"
    card_content += "To clear the previous conversation: 'Alexa, ask teachme to clear the progress'\n"
    
    welcome_text = "Welcome to Teach me skill. Which scenario do you want to learn? I will help you to practice it."
    reprompt_text = "I'm sorry - I didn't get it, could you please say that again?"

    return question(welcome_text).reprompt(reprompt_text).simple_card(title=card_title, content=card_content)

@ask.intent('AMAZON.CancelIntent')
def cancel_intent():
    return statement("Okay, good bye.")

@ask.intent('AMAZON.StopIntent')
def stop_intent():
    return statement("Okay, good bye.")

@ask.intent('AMAZON.HelpIntent')
def help_intent():
    card_title = "Help!"
    
    card_content += "To start a new scenario - e.g.'Order food'\n"
    card_content += "\n"
    card_content += "----------\n"
    card_content += "To end the skill - 'Alexa, exit/stop'\n"
    card_content += "To resume the conversation - 'Alexa, ask teachme to continue'\n"
    card_content += "To clear the previous conversation - 'Alexa, ask teachme to clear the progress'\n"
    
    return statement("Here is a list of command you could carry out.").simple_card(title=card_title, content=card_content)

@ask.intent("continue_intent")
def continue_intent():
    # get the previous conversation state from the database
    device_id = context.System.device.deviceId
    previous_data = get_item(device_id)
    intent_id = previous_data["intent_id"]
    intent_name = previous_data["intent_name"]
    corpus_name = previous_data["corpus_name"]
    
    # get data from the corpus
    corpus = Corpus(corpus_name)
    card_title = "Continue!"
    card_content = corpus.data[intent_id]["context"]
    card_content += "\n"
    card_content += "\n"
    card_content += corpus.data[intent_id]["card_text"]
    
    # add extra time for user to response
    reprompt_text = "Sorry, I didn't get it. Could you please say that again?"
    
    # push the card with an empty question
    return question("Okay, now look at the card").reprompt(reprompt_text).simple_card(title=card_title, content=card_content)
    
@ask.intent("clear_intent")
def clear_intent():
    # clear all entries from the database
    device_id = context.System.device.deviceId
    delete_item(device_id)
    
    # push the card with an statement
    card_title = "Clear conversation!"
    card_content = "The conversation is cleared.\nYou could invoke the skill again to start a new conversation."
    return statement("Okay, now the conversation is cleared").simple_card(title=card_title, content=card_content)

@ask.intent("restaurant_start_intent") # "0"
def restaurant_start_intent():
    # initialise the conversation state to the database - "0"
    device_id = context.System.device.deviceId
    intent_id = "0"
    intent_name = "restaurant_start_intent"
    corpus_name = "restaurant_corpus_test"
    
    update_item(device_id, intent_id, intent_name, corpus_name)

    # get data from the corpus
    corpus = Corpus(corpus_name)
    card_title = "Restaurant"
    card_content = corpus.data[intent_id]["context"]
    card_content += "\n"
    card_content += " "
    card_content += "\n"
    card_content += corpus.data[intent_id]["card_text"]
    alexa_response = corpus.data[intent_id]["alexa_response"]
    img_url = corpus.data[intent_id]["img_url"]
    
    # add hints for voice commands
    card_content += "\n\n----------\n"
    card_content += "To end the skill - 'Alexa, exit/stop'\n"
    card_content += "To resume the conversation - 'Alexa, ask teachme to continue'\n"
    card_content += "To clear the previous conversation - 'Alexa, ask teachme to clear the progress'\n"
    
    # add extra time for user to response
    reprompt_text = "Sorry, I didn't get it. Could you please say that again?"
    
    # get ready for the conversation
    start_template = "<speak>Loading corpus, please wait for 5 seconds. <break time='5s'/>, {}</speak>"
    
    # push the card with Alexa response from the current conversation state - "0"
    return question(start_template.format(alexa_response)).reprompt(reprompt_text).standard_card(title=card_title, text=card_content, 
                                                                                                 small_image_url=img_url, 
                                                                                                 large_image_url=img_url)
    
    
@ask.intent("restaurant_deliver_intent") # "1"
def restaurant_deliver_intent(food_name):
    if (food_name is None):
        response_text = "I can't recognise the food you have asked for, maybe try another food?"
        return question(response_text)
    
    # get the previous conversation state from the database - "0"
    device_id = context.System.device.deviceId
    previous_data = get_item(device_id)
    
    # check if the previous intent name equal to restaurant_start_intent
    previous_intent = previous_data["intent_name"]
    if previous_intent == "restaurant_start_intent":
        # update the current conversation state to the database - "1"
        intent_id = str(int(previous_data["intent_id"]) + 1)
        intent_name = "restaurant_deliver_intent"
        corpus_name = previous_data["corpus_name"]
        update_item(device_id, intent_id, intent_name, corpus_name)
    else: # do not update
        intent_id = previous_data["intent_id"]
        intent_name = previous_data["intent_name"]
        corpus_name = previous_data["corpus_name"]
    
    # get data from the corpus
    corpus = Corpus(corpus_name)
    card_title = "Restaurant"
    card_content = corpus.data[intent_id]["context"]
    card_content += "\n"
    card_content += " "
    card_content += "\n"
    card_content += corpus.data[intent_id]["card_text"]
    alexa_response = corpus.data[intent_id]["alexa_response"].format(food_name)
    img_urls = corpus.data[intent_id]["img_url"].split("|")
    
    # select the correct img
    img_url = ""
    import re
    for url in img_urls:
        if bool(re.search(food_name.split()[0], url)): # search for the first word of the food_name in img url
            img_url = url
            
    # add hints for voice commands
    card_content += "\n\n----------\n"
    card_content += "To end the skill - 'Alexa, exit/stop'\n"
    card_content += "To resume the conversation - 'Alexa, ask teachme to continue'\n"
    card_content += "To clear the previous conversation - 'Alexa, ask teachme to clear the progress'\n"
            
    # add extra time for user to response
    reprompt_text = "Sorry, I didn't get it. Could you please say that again?"
            
    # push the card with Alexa response from the current conversation state - "1"
    return question(alexa_response).reprompt(reprompt_text).standard_card(title=card_title, text=card_content, 
                                                                          small_image_url=img_url, 
                                                                          large_image_url=img_url)


@ask.intent("restaurant_soup_intent") # "2"
def restaurant_soup_intent():
    # get the previous conversation state to the database - "1"
    device_id = context.System.device.deviceId
    previous_data = get_item(device_id)
    
    # check if the previous intent name equal to restaurant_start_intent
    previous_intent = previous_data["intent_name"]
    if previous_intent == "restaurant_deliver_intent":
        # update the current conversation state to the database - "2"
        intent_id = str(int(previous_data["intent_id"]) + 1)
        intent_name = "restaurant_soup_intent"
        corpus_name = previous_data["corpus_name"]
        update_item(device_id, intent_id, intent_name, corpus_name)
    else: # do not update
        intent_id = previous_data["intent_id"]
        intent_name = previous_data["intent_name"]
        corpus_name = previous_data["corpus_name"]
    
    # get data from the corpus
    corpus = Corpus(corpus_name)
    card_title = "Restaurant"
    card_content = corpus.data[intent_id]["context"]
    card_content += "\n"
    card_content += " "
    card_content += "\n"
    card_content += corpus.data[intent_id]["card_text"]
    alexa_response = corpus.data[intent_id]["alexa_response"]
    img_url = corpus.data[intent_id]["img_url"]
    
    # add hints for voice commands
    card_content += "\n\n----------\n"
    card_content += "To end the skill - 'Alexa, exit/stop'\n"
    card_content += "To resume the conversation - 'Alexa, ask teachme to continue'\n"
    card_content += "To clear the previous conversation - 'Alexa, ask teachme to clear the progress'\n"
    
    # add extra time for user to response
    reprompt_text = "Sorry, I didn't get it. Could you please say that again?"
    
    # push the card with Alexa response from the current conversation state - "2"
    return question(alexa_response).reprompt(reprompt_text).standard_card(title=card_title, text=card_content, 
                                                                          small_image_url=img_url,
                                                                          large_image_url=img_url)

@ask.intent("restaurant_confirm_intent") # "3"
def restaurant_confirm_intent(decision):
    # get the previous conversation state to the database - "2"
    device_id = context.System.device.deviceId
    previous_data = get_item(device_id)
    
    # check if the previous intent name equal to restaurant_start_intent
    previous_intent = previous_data["intent_name"]
    if previous_intent == "restaurant_soup_intent":
        # update the current conversation state to the database - "3"
        intent_id = str(int(previous_data["intent_id"]) + 1)
        intent_name = "restaurant_confirm_intent"
        corpus_name = previous_data["corpus_name"]
        update_item(device_id, intent_id, intent_name, corpus_name)
    else: # do not update
        intent_id = previous_data["intent_id"]
        intent_name = previous_data["intent_name"]
        corpus_name = previous_data["corpus_name"]
    
    # get data from the corpus
    corpus = Corpus(corpus_name)
    card_title = "Restaurant"
    card_contexts = corpus.data[intent_id]["context"].split("|")
    alexa_responses = corpus.data[intent_id]["alexa_response"].split("|")
    img_urls = corpus.data[intent_id]["img_url"].split("|")
    
    # since it is the last intent of the conversation, clear the database
    delete_item(device_id)
    
    # select correct card content, alexa_response and img_url
    card_context = ""
    alexa_response = ""
    img_url = ""
    import re
    if bool(re.search(decision.split()[0].lower(), "yes")):
        card_context = card_contexts[0]
        alexa_response = alexa_responses[0]
        img_url = img_urls[0]
    else:
        card_context = card_contexts[-1]
        alexa_response = alexa_responses[-1]
        img_url = img_urls[-1]
        
    # update card content according to the decision
    card_content =  card_context + "\n"
    card_content += " "
    card_content += "\n"
    card_content += corpus.data[intent_id]["card_text"]
    
    # add hints for voice commands
    card_content += "\n\n----------\n"
    card_content += "To end the skill - 'Alexa, exit/stop'\n"
    card_content += "To resume the conversation - 'Alexa, ask teachme to continue'\n"
    card_content += "To clear the previous conversation - 'Alexa, ask teachme to clear the progress'\n"
    
    # add extra time for user to response
    reprompt_text = "Sorry, I didn't get it. Could you please say that again?"
    
    # get ready for the end of the conversation
    end_template = "<speak>{}, <break time='3s'/>. Thank you for practicing. This is the end of the conversation.</speak>"
    
    # push the card with Alexa response from the current conversation state - "3"
    return question(end_template.format(alexa_response)).reprompt(reprompt_text).standard_card(title=card_title, text=card_content, 
                                                                                               small_image_url=img_url,
                                                                                               large_image_url=img_url) # end of conversation

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)